In [14]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
from scipy import sparse
import scipy
import sys
sys.path.append('..')
from importlib import reload
import optm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [15]:
reload(optm)

<module 'optm' from '/Users/keisukemurota/Documents/todo/worms/python/nsp/optm.py'>

In [16]:
X = torch.rand((16,16),dtype=torch.float64)*2-1
X = X + X.T
model = optm.unitary_solver([4,4],syms=True)
model2 = optm.matrix_solver([4,4],syms=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0, weight_decay=0.05)

In [17]:
model2.matrix()

tensor([[2.7830, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.7892, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 3.3699, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.8783, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.7892, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.1503, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1665, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.00

RuntimeError: expected scalar type Double but found Float

In [16]:
for t in range(20000):
    y = model(X)
    loss = optm.loss_1(y)
    if t % 1000 == 0:
#         print(y)
        print(t, loss.item())
#         print(next(model.parameters()))
        print("\n")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 84.7134156907988


1000 61.74517501825012


2000 62.28437090820736


3000 63.76427491408256


4000 59.951676469127484


5000 59.712485909255754


6000 62.44161717696958


7000 63.12272788258092


8000 60.89974400270404


9000 61.59531851766686


10000 61.1471856247586


11000 61.482957268477065


12000 61.82753935739209


13000 62.54878470096159


14000 60.987553161376404


15000 59.44005098995243


16000 59.218723968281914


17000 60.996570847775686


18000 61.73681213815203


19000 59.599999336746464




In [17]:
# u = model.one_site_matrix.data
# U = torch.kron(u,u)
U = model.matrix

In [18]:
tmp = U @ X @ U.T
torch.where(tmp < 0)

(tensor([ 0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,
          2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  5,  5,  5,
          5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,
          8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10,
         10, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13,
         13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15]),
 tensor([ 0,  1,  4,  5,  9, 15,  0,  1,  3,  6, 10, 12, 13,  6,  8,  9, 10, 12,
         14, 15,  1,  3,  5,  6,  7,  8, 12, 15,  0,  8, 10, 12, 13,  0,  3,  5,
          6,  7,  8, 14,  1,  2,  3,  5,  6,  9, 10, 11, 14, 15,  3,  5, 11, 12,
          2,  3,  4,  5,  8, 15,  0,  2,  6, 12, 13, 15,  1,  2,  4,  6, 11, 13,
         15,  6,  7, 10, 11,  1,  2,  3,  4,  7,  9, 13, 14,  1,  4,  9, 10, 12,
         15,  2,  5,  6, 12, 15,  0,  2,  3,  6,  8,  9, 10, 13, 14]))

In [19]:
tmp

tensor([[-1.2637e+00, -5.9350e-01,  3.2222e-01,  4.2895e-03, -1.7757e-01,
         -1.3073e-01,  4.1947e-01,  1.0912e+00,  5.3388e-01, -1.2388e+00,
          2.7204e-01,  8.0699e-02,  3.4655e-01,  1.2000e+00,  1.5556e+00,
         -1.5535e+00],
        [-5.9350e-01, -5.7445e-01,  1.6736e+00, -1.0549e-01,  6.3196e-01,
          7.1800e-01, -7.1169e-03,  6.4767e-01,  2.0338e-01,  9.0001e-01,
         -6.2547e-02,  1.9446e-01, -1.8408e+00, -8.8294e-02,  1.3914e-01,
          1.2739e+00],
        [ 3.2222e-01,  1.6736e+00,  7.6978e-01,  3.4935e-01,  1.0349e+00,
          4.8983e-01, -4.9617e-01,  1.7991e+00, -8.7214e-01, -2.6135e-01,
         -6.5645e-02,  1.3359e-01, -2.9210e-01,  8.0559e-01, -1.7937e-01,
         -3.9844e-01],
        [ 4.2895e-03, -1.0549e-01,  3.4935e-01, -6.2874e-01,  8.6135e-01,
         -2.5092e-02, -3.2712e-01, -4.5880e-02, -1.7703e-01,  9.6211e-02,
          1.9591e-01,  5.0951e-01, -8.2023e-01,  1.1158e+00,  2.6127e-01,
         -1.1396e-01],
        [-1.7757e-01

In [22]:
torch.diag(model._params)

tensor([[ 0.2502,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.0757,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0721,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.4275,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.8681,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0988]],
       grad_fn=<DiagBackward0>)